<a href="https://colab.research.google.com/github/magno12345/UPSO/blob/main/TP1_Analisis_Ventas_Mensuales_Magno_Sotelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Carga y limpieza de datos

In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import datetime

# Lista de nombres de meses
meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

# Crear una lista para almacenar los DataFrames de cada mes
dataframes_meses = []

# Ruta base de los archivos CSV
ruta_base = '/content/drive/MyDrive/DATASETS BIG DATA/Dataset_de_ventas_'

# Iterar a través de los meses y cargar los datos en un DataFrame
for mes in meses:
    # Componer la ruta completa del archivo CSV
    ruta_csv = f'{ruta_base}{mes}.csv'

    # Leer el archivo CSV y asignar el nombre del mes
    df_mes = pd.read_csv(ruta_csv).assign(Mes=mes)


    # Convertir columnas numéricas  Cantidad Pedida y Precio Unitario a tipos de datos numéricos
    df_mes['Cantidad Pedida'] = pd.to_numeric(df_mes['Cantidad Pedida'], errors='coerce')
    df_mes['Precio Unitario'] = pd.to_numeric(df_mes['Precio Unitario'], errors='coerce')
    df_mes['ID de Pedido'] = pd.to_numeric(df_mes['ID de Pedido'], errors='coerce').astype("Int64")



    # Eliminar filas con valores nulos
    df_mes = df_mes.dropna()


    # Agregar el DataFrame del mes a la lista
    dataframes_meses.append(df_mes)



# Concatenar todos los DataFrames en uno solo
df_total = pd.concat(dataframes_meses, ignore_index=True)



# Transformo los valores de fecha y hora a un formato que se pueda analizar
df_total['Fecha de compra'] = pd.to_datetime(df_total['Fecha de Pedido'])
#le agrego las columnas hora y dia
df_total['Hora'] =  df_total['Fecha de compra'].dt.hour
df_total['Dia'] = df_total['Fecha de compra'].dt.day
#Transformo todos los valores numericos al tipo de dato int
df_total['Cantidad Pedida'] = df_total['Cantidad Pedida'].astype(int)
df_total['Precio Unitario'] = df_total['Precio Unitario'].astype(float)

df_total





,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Mes,Fecha de compra,Hora,Dia
0,141234,iPhone,1,700.00,01/22/19 21:25,"944 Walnut St, Boston, MA 02215",Enero,2019-01-22 21:25:00,21,22
1,141235,Lightning Charging Cable,1,14.95,01/28/19 14:15,"185 Maple St, Portland, OR 97035",Enero,2019-01-28 14:15:00,14,28
2,141236,Wired Headphones,2,11.99,01/17/19 13:33,"538 Adams St, San Francisco, CA 94016",Enero,2019-01-17 13:33:00,13,17
3,141237,27in FHD Monitor,1,149.99,01/05/19 20:33,"738 10th St, Los Angeles, CA 90001",Enero,2019-01-05 20:33:00,20,5
4,141238,Wired Headphones,1,11.99,01/25/19 11:59,"387 10th St, Austin, TX 73301",Enero,2019-01-25 11:59:00,11,25
...,...,...,...,...,...,...,...,...,...,...
185945,319666,Lightning Charging Cable,1,14.95,12/11/19 20:58,"14 Madison St, San Francisco, CA 94016",Diciembre,2019-12-11 20:58:00,20,11
185946,319667,AA Batteries (4-pack),2,3.84,12/01/19 12:01,"549 Willow St, Los Angeles, CA 90001",Diciembre,2019-12-01 12:01:00,12,1
185947,319668,Vareebadd Phone,1,400.00,12/09/19 06:43,"273 Wilson St, Seattle, WA 98101",Diciembre,2019-12-09 06:43:00,6,9
185948,319669,Wired Headphones,1,11.99,12/03/19 10:39,"778 River St, Dallas, TX 75001",Diciembre,2019-12-03 10:39:00,10,3


##1. Comportamiento de las ventas en los distintos meses:




*  ¿Cómo variaron las ventas a lo largo de los diferentes meses?
*  ¿Hubo algún mes que se destacó en términos de ventas?




In [21]:
df_ventas_por_mes = df_total.groupby("Mes")["Cantidad Pedida"].sum().reset_index()

import plotly.express as px
import plotly.graph_objects as go


orden_meses = ["Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio", "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"]

# Reordenar los datos en df_ventas_por_mes según el orden de los meses
df_ventas_por_mes['Mes'] = pd.Categorical(df_ventas_por_mes['Mes'], categories=orden_meses, ordered=True)
df_ventas_por_mes = df_ventas_por_mes.sort_values('Mes')

# Suponiendo que tienes un DataFrame df_ventas_por_mes con la columna "Mes" y "Cantidad Pedida"
fig = px.line(
    df_ventas_por_mes,
    x='Mes',
    y='Cantidad Pedida',
    title='Variación de las Ventas por Mes',
    labels={'Mes': 'Mes', 'Cantidad Pedida': 'Ventas'},
    template='plotly',  # Estilo del gráfico
    markers=True,#esto me resalta los puntos en el grafico.
)

# Encontrar el mes con las ventas máximas
mes_max_ventas = df_ventas_por_mes[df_ventas_por_mes['Cantidad Pedida'] == df_ventas_por_mes['Cantidad Pedida'].max()]['Mes'].values[0]

# Agregar una marca de cruz roja en el punto del mes con las ventas máximas
fig.add_trace(
    go.Scatter(
        x=[mes_max_ventas],
        y=[df_ventas_por_mes['Cantidad Pedida'].max()],
        mode='markers',
        marker=dict(symbol='x', color='red', size=12),
        showlegend=False,
        name='Ventas máximas',# Cambiar el nombre en la leyenda
    )
)




# Modificar la escala en el eje x para mostrar valores completos
fig.update_yaxes(tickformat='- d')

# Modificar el tamaño de la fuente del título y los ejes
fig.update_layout(
    title_font=dict(size=20),  # Tamaño del título
    xaxis=dict(title_font=dict(size=20),tickfont=dict(size=17)),  # Tamaño del título del eje x
    yaxis=dict(title_font=dict(size=20),tickfont=dict(size=17)),  # Tamaño del título del eje y
)
# Mostrar el gráfico interactivo
fig.show()


##Pregunta Adicional:Variacion Mensual

In [22]:
#creo un nuevo df
df_var_men = df_ventas_por_mes
# Calcular la variación mensual
df_var_men['Variacion_Mensual'] = df_var_men['Cantidad Pedida'].pct_change() * 100
#le agrego un 0 a enero xq no tiene mes anterior y no se puede calcular
df_var_men['Variacion_Mensual'].fillna(0, inplace=True)  # Rellenar NaN con 0

#  orden cronológico de los meses
orden_mesess = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']


# Crear un gráfico de barras con paleta de colores divergente
fig = px.bar(df_var_men, x='Mes', y='Variacion_Mensual', title='Variación Mensual de las Ventas',
             color='Variacion_Mensual', color_continuous_scale='plasma', category_orders={"Mes": orden_mesess})

# Personalizar el gráfico

fig.update_layout(xaxis_title='Mes', yaxis_title='Variación Mensual (%)')
#La variacion mensual ayuda a entender como cambian la ventas de un mes a otro,es util para detectar patrones.

fig.show()



¿Cual es el ingreso total generado x mes?

In [23]:
import plotly.graph_objects as go

#df_ingreso_total = df_total[['Precio Unitario',"Mes"]]#TOMO LAS COLUMNAS QUE NECESITO NOMAS
df_ingreso_total = df_total.groupby("Mes")["Precio Unitario"].sum().reset_index().round(0).sort_values(by="Precio Unitario",ascending=False)#agrupo x mes y al precio unitario lo sumo y lo redondeo


# Crear una columna categórica con el orden de los meses
df_ingreso_total['Mes'] = pd.Categorical(df_ingreso_total['Mes'], categories=orden_meses, ordered=True)

# Ordenar los datos por la columna categórica "Mes"
df_ingreso_total = df_ingreso_total.sort_values('Mes')


ingreso_total = df_ingreso_total["Precio Unitario"]
meses = df_ingreso_total["Mes"]

fig = px.bar(x = meses, y = ingreso_total, labels={'x':'Meses', 'y':'Ingreso Total'})
fig.update_layout(title='Ingreso Total por Mes')




fig = go.Figure()
fig.add_trace(go.Bar(
    x=meses,
    y=ingreso_total,
    text=ingreso_total,  #  voy a utilizar ingreso_total como texto en las barras
    textposition='auto',  # Coloco automáticamente el texto en la parte superior de las barras
    marker_color='light blue',  # Cambio el color de las barras
))

fig.update_layout(
    title='Ingreso Total por Mes',
    xaxis_title='Meses',
    yaxis_title='Ingreso Total',
)


#df_ingreso_total
fig.show()

##2. Optimización de la publicidad y patrón de ventas por hora:

¿Cuál es el momento ideal del día para presentar la publicidad y aumentar la
probabilidad de compra?

In [24]:
df_copia = df_total.copy()

df_hr = df_copia.groupby( "Hora" )["Cantidad Pedida"].sum().reset_index()
hora = df_hr["Hora"]
cantidad = df_hr["Cantidad Pedida"]
me = df_copia["Mes"].drop_duplicates()
#df = df_copia.groupby( by = "Producto")["Cantidad Pedida"].count().reset_index()

import plotly.graph_objects as go

#Primero Crear una figura
fig = go.Figure()

# Agrego al objeto figura un grafico de de linea con las horas y las ventas por hora
fig.add_trace(go.Scatter(

    x = hora,
    y = cantidad,

    mode='lines',
    name="ventas",
    line=dict(color="red")))

# Personalizar el diseño del gráfico
fig.update_layout(
    title='Gráfico de Hora ideal para la publicidad ',
    xaxis=dict(
        title='Hora',
        tickvals=list(range(24)),  # Valores del eje X de 0 a 23
        ticktext=[str(i) for i in range(24)],),
    yaxis=dict(title='Cantidad Pedida',tickformat='-d')#para que me muestre los valores en numeros enteros.
)


# Mostrar el gráfico
fig.show()


¿Cómo cambian los patrones de ventas por hora a lo largo del año?

In [25]:
df_copia = df_total.copy()

df_hr = df_copia.groupby(["Mes","Hora"])["Cantidad Pedida"].sum().reset_index()

horas = df_hr["Hora"]
# Crear un gráfico de líneas
fig = px.line(df_hr, x = "Hora" , y =  "Cantidad Pedida",color="Mes" ,line_shape='spline',markers=True)#cambio el estilo de linea

# Configuro los ejes y los títulos
fig.update_layout(
    title = 'Grafico de patrones de venta por hora en el año',
    xaxis = dict(title='Horas', tickvals = horas),
    yaxis = dict(title='Unidades Vendidas' , side="left"),
    )
fig.show()




##3. Distribución de ventas por ubicación:

¿En qué ciudades se han registrado las mayores ventas?(ingresos)

In [26]:
#renombro la columna direccion de envio para trabajar mejor y reemplazo los meses de string a numero .
df_ciudad = df_total.copy().rename(columns={"Dirección de Envio":"Direccion de Envio"}).replace(["Enero", "Febrero", "Marzo",
                                                                                                 "Abril","Mayo","Junio",
                                                                                                 "Julio","Agosto","Septiembre",
                                                                                                 "Octubre","Noviembre","Diciembre"],
                                                                                                  [1,2,3,4,5,6,7,8,9,10,11,12])#le cambio el nombre x el acento

#Creo una nueva columna "ciudad" y de la columna direccion de envio ,me quedo con el indice 1 . a cada elemento se separa por coma .
df_ciudad["ciudad"]=df_ciudad["Direccion de Envio"].apply(lambda x: x.split(",")[1])
#Creo la nueva columna total ventas para calcular el precio final del producto
df_ciudad["total_ventas"] = df_ciudad["Cantidad Pedida"] * df_ciudad["Precio Unitario"]
#Calculo en que ciudad hubo mayores y menores cantidades pedidas
df_ciudad_mayor_ingre = df_ciudad.groupby("ciudad")["Cantidad Pedida"].sum().reset_index().sort_values("Cantidad Pedida",ascending = True)
city =df_ciudad_mayor_ingre["ciudad"]
cantidad_ventas=df_ciudad_mayor_ingre["Cantidad Pedida"]
#Ahora lo grafico creando una figura
fig = px.bar(df_ciudad_mayor_ingre, x = cantidad_ventas, y = city)
#Actualizo algunos datos de la figura
fig.update_layout(
    title='Ciudades con las Mayores ingresos',
    xaxis_title='Ingresos Totales',
    yaxis_title='Ciudades',
)
fig.show()


¿En qué ciudades se han registrado las mayores ventas?

In [27]:
df_mayor_venta = df_ciudad.groupby(["Mes","ciudad"])["Cantidad Pedida"].sum().reset_index()

# Cambio los numeros de los meses a string para mejor visualizacion
meses = {1: 'Enero', 2: 'Febrero', 3: 'Marzo', 4: 'Abril', 5: 'Mayo', 6: 'Junio', 7: 'Julio', 8: 'Agosto', 9: 'Septiembre', 10: 'Octubre', 11: 'Noviembre', 12: 'Diciembre'}

# Convierto los números de mes en nombres de mes
df_mayor_venta['Mes'] = df_mayor_venta['Mes'].map(meses)

# Crear un gráfico de líneas
fig = px.line(df_mayor_venta, x = df_mayor_venta["ciudad"] , y =  df_mayor_venta["Cantidad Pedida"] , color="Mes",line_shape='spline',markers=True)#cambio el estilo de linea
fig.show()



¿Cómo se comparan las ventas en diferentes estados o regiones?

In [28]:
#Uso la biblioteca "re" para buscar patrones especifico en la cadena de texto
def region(columna):
    estado_match = re.search(r'\b[A-Z]{2}\b', columna)
    if estado_match:
        estado = estado_match.group()
        return estado
    else:
        return None

#Creo una columna region q contendra los caracteres de los estados/regiones
df_ciudad["Region"] = df_ciudad["Direccion de Envio"].apply(region)

df_region = df_ciudad.groupby("Region")["Cantidad Pedida"].sum().reset_index().sort_values("Cantidad Pedida")

fig = px.choropleth(df_region, locations='Region', locationmode='USA-states',color='Cantidad Pedida',
                           color_continuous_scale="RdYlGn",
                           scope="usa",
                          title= 'Ventas por estado'
                          )

fig.show()
#df_region

¿Existe variación en las ventas por estado a lo largo de los meses?

In [29]:
df_var = df_ciudad.groupby(["Region","Mes"])["total_ventas"].sum().reset_index()

# Crear un gráfico de líneas
fig = px.line(df_var, x='Mes', y='total_ventas', color='Region',
              labels={'total_ventas': 'Total de Ventas', 'Mes': 'Mes'},
              title='Variación de Ventas por Región a lo largo de los Meses')

fig.update_xaxes(type='category')  #es  para asegurarme de que los meses se muestren en orden



fig.show()


##4. Análisis del producto más vendido:

¿Cuál es el producto más vendido en general y en cada mes?

In [30]:
#Uso la funcion max() para encontrar el prod mas vendido en cada mes,si uso sum() es  para encontrar la cantidad máxima vendida de cada producto en cada mes.
df_producto_mas_vendido = df_ciudad.groupby(["Mes","Producto"])["Cantidad Pedida"].max().reset_index()
#busco el  indice con mayor valor en cada mes
indices_max_por_mes = df_producto_mas_vendido.groupby('Mes')['Cantidad Pedida'].idxmax()
#ahora uso los indices anteriores y selecciono las filas con iloc y lo reseteo para que se vea bien
productos_mas_vendidos_por_mes = df_producto_mas_vendido.loc[indices_max_por_mes].reset_index(drop=True)

productos_mas_vendidos_por_mes


,Mes,Producto,Cantidad Pedida
0,1,AAA Batteries (4-pack),7
1,2,AAA Batteries (4-pack),7
2,3,AAA Batteries (4-pack),7
3,4,AAA Batteries (4-pack),7
4,5,AAA Batteries (4-pack),7
5,6,AAA Batteries (4-pack),9
6,7,AAA Batteries (4-pack),9
7,8,AAA Batteries (4-pack),8
8,9,AA Batteries (4-pack),6
9,10,AAA Batteries (4-pack),8


¿Qué factores crees que han contribuido al éxito de ese producto en particular?

Baterias AAA

In [31]:
#filtro,y me llevo las celdas donde aparezcan las pilas AAA
df_factor = df_ciudad[(df_ciudad["Producto"] == "AAA Batteries (4-pack)")]

df_factor_fil = df_factor.groupby(["Dia","Mes"])["Cantidad Pedida"].sum().reset_index()
#si lo uso de este modo de abajo ,me dara problemas al pasar el cursor sobre la linea ,me aparecera asi x=15.
dia = df_factor_fil["Dia"]#solo dejo dia para que me de la escala de los 31 dias.
#ca = df_factor_fil["Cantidad Pedida"]

# Cambio los numeros de los meses a string para mejor visualizacion
meses = {1: 'Enero', 2: 'Febrero', 3: 'Marzo', 4: 'Abril', 5: 'Mayo', 6: 'Junio', 7: 'Julio', 8: 'Agosto', 9: 'Septiembre', 10: 'Octubre', 11: 'Noviembre', 12: 'Diciembre'}

# Convierto los números de mes en nombres de mes
df_factor_fil['Mes'] = df_factor_fil['Mes'].map(meses)


# Crear un gráfico de líneas
fig = px.line(df_factor_fil, x = df_factor_fil["Dia"] , y =  df_factor_fil["Cantidad Pedida"] , color="Mes",line_shape='spline')#cambio el estilo de linea

# Configuro los ejes y los títulos
fig.update_layout(
    title = "Grafico de Variacion de pedidos de Baterias AAA por dia en todos los meses",
    xaxis = dict(title='Dias', tickvals=dia),  # Para que en el eje x se muestren todos los valores uso tickvals=Horas
    yaxis = dict(title='Unidades Pedidas de pilas AAA',side='left'),
)
#busque que paso en diciembre porque se ve que hubo muchas compras durante ese mes,tubo un pico el 17 de diciembre de 2019,pero
#no hubo un hecho q ameritaba una compra grande de pilas aaa ,si hubo dias festivos



fig.show()

Por ciudad y region baterias AA(opcional)

In [32]:
df_fil2 =  df_factor.groupby(["Region","ciudad"])["Cantidad Pedida"].sum().reset_index()
# Crear un gráfico de líneas
fig = px.bar(df_fil2, x = df_fil2["Region"] , y =  df_fil2["Cantidad Pedida"],color="ciudad",barmode="stack")#cambio el estilo de linea

# Configuro los ejes y los títulos
fig.update_layout(
    title = "Gráfico de Barras Apiladas de Unidades Pedidas de Pilas AAA por Ciudad y Región",

    yaxis = dict(title='Unidades Pedidas de pilas AAA',side='left'),
)


fig.show()

Baterias AA

In [33]:
#filtro,y me llevo las celdas donde aparezcan las pilas AA
df_factor_2 = df_ciudad[(df_ciudad["Producto"] == "AA Batteries (4-pack)")]

df_factor_aa = df_factor_2.groupby(["Dia","Mes"])["Cantidad Pedida"].sum().reset_index()

# Convierto los números de mes en nombres de mes
df_factor_aa['Mes'] = df_factor_aa['Mes'].map(meses).copy()

# Crear un gráfico de líneas
fig = px.line(df_factor_aa, x = df_factor_aa["Dia"] , y =  df_factor_aa["Cantidad Pedida"] , color="Mes",line_shape='spline')#cambio el estilo de linea


# Configuro los ejes y los títulos
fig.update_layout(
    title = "Grafico de Variacion de pedidos de Baterias AA por dia en todos los meses",
    xaxis = dict(title='Dias', tickvals=dia),  # Para que en el eje x se muestren todos los valores uso tickvals=Horas
    yaxis = dict(title='Unidades Pedidas de pilas AA',side='left'),
)


fig.show()

Por ciudad y region baterias AA(opcional)

In [34]:
df_fil3 =  df_factor_2.groupby(["Region","ciudad"])["Cantidad Pedida"].sum().reset_index()
# Crear un gráfico de líneas
fig = px.bar(df_fil3, x = df_fil3["Region"] , y =  df_fil3["Cantidad Pedida"],color="ciudad",barmode="stack")#cambio el estilo de linea

# Configuro los ejes y los títulos
fig.update_layout(
    title = "Gráfico de Barras Apiladas de Unidades Pedidas de Pilas AA por Ciudad y Región",

    yaxis = dict(title='Unidades Pedidas de pilas AA',side='left'),
)


fig.show()

##5. Tendencia ventas:

¿Existe alguna tendencia o patrón en las ventas que se repita a lo largo de los días
en los diferentes meses?


In [35]:
df_pat_ventas = df_ciudad.groupby(["Dia","Mes"])["Cantidad Pedida"].sum().reset_index()
# Convierto los números de mes en nombres de mes
df_pat_ventas['Mes'] = df_pat_ventas['Mes'].map(meses)

# Crear un gráfico de líneas
fig = px.line(df_pat_ventas, x = df_pat_ventas["Dia"] , y =  df_pat_ventas["Cantidad Pedida"] , color="Mes",line_shape='spline')#cambio el estilo de linea


# Configuro los ejes y los títulos
fig.update_layout(
    title = "Grafico de Patron de Tendencias a lo largo de los meses y dias",
    xaxis = dict(title='Dias', tickvals=dia),  # Para que en el eje x se muestren todos los valores uso tickvals=Horas
    yaxis = dict(title='Unidades Pedidas',side='left'),
)




fig.show()



*  ¿Cómo varían las ventas a lo largo de los diferentes días de la semana?




In [36]:
# Función para extraer la fecha de una cadena en formato "MM/DD/YY HH:mm"
def extraer_fecha(fecha_hora_str):
    partes = fecha_hora_str.split()
    fecha_str = partes[0]
    fecha_obj = datetime.datetime.strptime(fecha_str, "%m/%d/%y")
    nombre_dia_semana = fecha_obj.strftime("%A")
    return nombre_dia_semana

# Aplico la función a la columna 'Fecha de Pedido' y creo una nueva columna 'DiaSemana'
df_ciudad['DiaSemana'] = df_ciudad['Fecha de Pedido'].apply(extraer_fecha)



# Defino el orden personalizado de los días de la semana para mejor visualizacion.
dias_semana_orden = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

# Convertir la columna 'DiaSemana' en una categoría con el orden personalizado
df_ciudad['DiaSemana'] = pd.Categorical(df_ciudad['DiaSemana'], categories=dias_semana_orden, ordered=True)

df_dia_sem = df_ciudad.groupby(["Mes","DiaSemana"])["Cantidad Pedida"].sum().reset_index()

# Convierto los números de mes en nombres de mes
df_dia_sem['Mes'] = df_dia_sem['Mes'].map(meses)

# Crear un gráfico de líneas
fig = px.line(df_dia_sem, x = df_dia_sem["DiaSemana"] , y =  df_dia_sem["Cantidad Pedida"], color="Mes",line_shape='spline',markers=True)#cambio el estilo de linea



# Configuro los ejes y los títulos
fig.update_layout(
    title = "Grafico de Variacion de ventas en los 7 dias de la semana",
    xaxis = dict(title='Dias de semana', tickvals=dias_semana_orden),  # Para que en el eje x se muestren todos los valores uso tickvals=Horas
    yaxis = dict(title='Unidades Pedidas',side='left'),
)
# Cambio los nombres de los días de la semana en el eje x a español
fig.update_xaxes(tickvals=dias_semana_orden, ticktext=["Lunes", "Martes", "Miércoles", "Jueves", "Viernes", "Sábado", "Domingo"])




fig.show()


¿Se observa alguna diferencia significativa en las ventas entre los días laborables y
los fines de semana?

In [37]:
import pandas as pd
import datetime
import plotly.express as px

# Función para extraer la fecha de una cadena en formato "MM/DD/YY HH:mm"
def extraer_fecha(fecha_hora_str):
    partes = fecha_hora_str.split()
    fecha_str = partes[0]
    fecha_obj = datetime.datetime.strptime(fecha_str, "%m/%d/%y")
    nombre_dia_semana = fecha_obj.strftime("%A")
    return nombre_dia_semana

# Aplico la función a la columna 'Fecha de Pedido' y creo una nueva columna 'DiaSemana'
df_ciudad['DiaSemana'] = df_ciudad['Fecha de Pedido'].apply(extraer_fecha)

# Defino los días laborables
dias_laborables = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]

# Creo una columna 'EsLaborable' basada en si el día es laborable o no
df_ciudad['EsLaborable'] = df_ciudad['DiaSemana'].isin(dias_laborables)

# Agrupo los datos por mes y si es laborable o no, calculando el promedio de las ventas
df_dia_sem = df_ciudad.groupby(["Mes", "EsLaborable"])["Cantidad Pedida"].mean().reset_index()

# Convierto los números de mes en nombres de mes
df_dia_sem['Mes'] = df_dia_sem['Mes'].map(meses)

# Creo un gráfico de líneas
fig = px.line(df_dia_sem, x="Mes", y="Cantidad Pedida", color="EsLaborable",
              labels={"Mes": "Mes", "Cantidad Pedida": "Promedio de Ventas"},
              color_discrete_map={True: "green", False: "orange"},
              title="Tendencia de Ventas Promedio en Días Laborables y No Laborables por Mes",line_shape="spline")

# Configuración adicional para mejorar la visualización
fig.update_yaxes(title='Unidades Pedidas', side='left')

# Personalizo la leyenda
fig.for_each_trace(lambda t: t.update(name="Días Laborables" if t.name == True else "Días No Laborables"))


fig.show()


##6. Impacto de eventos especiales en las ventas:


¿Se ha observado algún aumento o disminución significativa en las ventas en días
cercanos a eventos especiales, como días festivos?(informe)

¿Qué eventos específicos han tenido un impacto notable en el comportamiento de
las ventas y cómo se manifestó ese impacto?(informe)